#### Librerias

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from time import sleep
import csv

#### Mostramos la cantidad de comentarios que hay en la pagina principal

In [2]:
# Inicia el WebDriver
driver = webdriver.Chrome()

url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
driver.get(url)

try:
    review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]') #article[aria-roledescription="Review"], p[role="presentation"]
    
    # Verificá cuántos artículos se encontraron
    print(f"Se encontraron {len(review_articles)} artículos de reseña.")
    
except Exception as e:
    print(f"No se pudieron encontrar artículos de reseñas: {e}")
    driver.quit()
    exit()

Se encontraron 15 artículos de reseña.


#### Mostramos la cantidad de comentarios que hay en la pagina principal haciendo Scroll

In [3]:
# Inicia el WebDriver
driver = webdriver.Chrome()

url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
driver.get(url)


sleep(5) 

# Número total de reseñas
total_reviews = 0
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Busca los comentarios
    review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]')
    total_reviews = len(review_articles)
    print(f"Total de reseñas encontradas: {total_reviews}")

    # Hace el scroll en la web
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    sleep(2)

    # Verificá si llego al final de la pagina
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

print(f"Total final de reseñas: {total_reviews}")
driver.quit()
exit()


Total de reseñas encontradas: 15
Total de reseñas encontradas: 30
Total de reseñas encontradas: 45
Total de reseñas encontradas: 60
Total de reseñas encontradas: 75
Total de reseñas encontradas: 90
Total de reseñas encontradas: 105
Total de reseñas encontradas: 120
Total de reseñas encontradas: 135
Total de reseñas encontradas: 150
Total de reseñas encontradas: 165
Total de reseñas encontradas: 180
Total de reseñas encontradas: 195
Total de reseñas encontradas: 210
Total final de reseñas: 210


: 

## Desarrolló
Logramos ver que los comentarios que aparecian en la pagina principal no eran absolutamente todos, así que decidimos hacer un scraping por la categoria de comentarios, es decir, un scraping que comience por la categoria de 5 estrellas hasta la categoria de 1 una estrella, por eso comenzaremos con la interacción del botón de catacteristicas

#### Comenzamos la interacción con el botón "Calificaciones"

In [4]:
# # Inicia el WebDriver
# driver = webdriver.Chrome()

# url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
# driver.get(url)

# # busca el boton de calificaciones y hace clik
# try:
#     mostrar_mas_button = driver.find_element(By.XPATH, '//button[@id=":R1inbh:"]') #//button[@data-testid='see-more']    
#     mostrar_mas_button.click()
#     print("Botón de 'Mostrar más comentarios' fue clickeado correctamente.")
    
#     (5)
    
# except Exception as e:
#     print(f"No se pudo hacer clic en el botón: {e}")
#     driver.quit()
#     exit()


Botón de 'Mostrar más comentarios' fue clickeado correctamente.


: 

In [12]:
# Inicializar el WebDriver
driver = webdriver.Chrome()

# Navegar a la página de producto en Mercado Libre
url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
driver.get(url)

sleep(10)

# Definir las categorías de calificación
ratings = ['5', '4', '3', '2', '1']

# Recorrer cada calificación
for rating in ratings:
    while True:  # Bucle que intentará seleccionar la calificación
        # Hacer clic en el botón de calificación
        rating_button = driver.find_element(By.XPATH, '//button[@id=":R1inbh:"]')
        rating_button.click()
        sleep(5)

        # Esperar a que la opción de calificación esté presente y sea visible
        try:
            rating_option = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, f'//li[@data-testid="filterItem-rating-{rating}"]'))
            )
            rating_option.click()
            sleep(10)  # Esperar a que carguen los comentarios

            # Contar los comentarios
            review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]')
            total_reviews = len(review_articles)
            print(f"Total de reseñas con {rating} estrella(s): {total_reviews}")

            break  # Salir del bucle si la calificación se encuentra

        except TimeoutException:
            print(f"No se pudo encontrar la opción de {rating} estrella(s).")
            break  # Salir del bucle si la opción no está disponible

# Cerrar el navegador
driver.quit()


Total de reseñas con 5 estrella(s): 15
Total de reseñas con 4 estrella(s): 15
Total de reseñas con 3 estrella(s): 15
Total de reseñas con 2 estrella(s): 15
Total de reseñas con 1 estrella(s): 15


In [17]:
# # Inicializar el WebDriver
# driver = webdriver.Chrome()

# # Navegar a la página de producto en Mercado Libre
# url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
# driver.get(url)

# sleep(5)

# # Definir las categorías de calificación
# ratings = ['1', '2', '3', '4', '5']

# # Recorrer cada calificación
# for rating in ratings:
#     # Hacer clic en el botón de calificación
#     rating_button = driver.find_element(By.XPATH, f'//button[@id=":R1inbh:"]')
#     rating_button.click()
#     sleep(7)

#     # Seleccionar la calificación deseada
#     rating_option = driver.find_element(By.XPATH, f'//li[@data-testid="filterItem-rating-{rating}"]')
#     rating_option.click()
#     sleep(7)  # Esperar a que carguen los comentarios

#     # Contar los comentarios
#     review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]')
#     total_reviews = len(review_articles)
#     print(f"Total de reseñas con {rating} estrella(s): {total_reviews}")

#     # Volver a abrir el menú de calificación para seleccionar otra calificación
#     rating_button.click()
#     sleep(7)

# # Cerrar el navegador
# driver.quit()


In [13]:
# # Inicializar el WebDriver
# driver = webdriver.Chrome()

# # Navegar a la página de producto en Mercado Libre
# url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
# driver.get(url)

# sleep(5)

# # # Definir las categorías de calificación
# # ratings = ['1', '2', '3', '4', '5']

# # Recorrer cada calificación
# # for rating in ratings:

# # Hacer clic en el botón de calificación
# rating_button = driver.find_element(By.XPATH, '//button[@id=":R1inbh:"]')
# rating_button.click()
# sleep(2)

# # Esperar a que la opción de calificación esté presente y sea visible
# try:
#     rating_option = WebDriverWait(driver, 10).until(
#         EC.visibility_of_element_located((By.XPATH, '//li[@data-testid="filterItem-rating-2"]'))
#     )
#     rating_option.click()
#     sleep(5)  # Esperar a que carguen los comentarios

#     # Contar los comentarios
#     review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]')
#     total_reviews = len(review_articles)
#     print(f"Total de reseñas con {rating} estrella(s): {total_reviews}")

#     # # Volver a abrir el menú de calificación para seleccionar otra calificación
#     # rating_button.click()
#     # sleep(2)

# except TimeoutException:
#     print(f"No se pudo encontrar la opción de  estrella(s).")

# # Cerrar el navegador
# driver.quit()


In [14]:
# # Inicializar el WebDriver
# driver = webdriver.Chrome()

# # Navegar a la página de producto en Mercado Libre
# url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
# driver.get(url)

# sleep(10)

# # Definir las categorías de calificación ['5', '4', '3', '2', '1']
# ratings = ['5', '5','4','4','3','3', '2','2', '1', '1']

# # Recorrer cada calificación
# for rating in ratings:
#     # Hacer clic en el botón de calificación
#     rating_button = driver.find_element(By.XPATH, '//button[@id=":R1inbh:"]')
#     rating_button.click()
#     sleep(5)

#     # Esperar a que la opción de calificación esté presente y sea visible
#     try:
#         rating_option = WebDriverWait(driver, 10).until(
#             EC.visibility_of_element_located((By.XPATH, f'//li[@data-testid="filterItem-rating-{rating}"]'))
#         )
#         rating_option.click()
#         sleep(10)  # Esperar a que carguen los comentarios

#         # Contar los comentarios
#         review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]')
#         total_reviews = len(review_articles)
#         print(f"Total de reseñas con {rating} estrella(s): {total_reviews}")

#         # Volver a abrir el menú de calificación para seleccionar otra calificación
#         rating_button.click()
#         sleep(5)

#     except TimeoutException:
#         print(f"No se pudo encontrar la opción de {rating} estrella(s).")

# # Cerrar el navegador
# driver.quit()
